In [91]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [92]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [93]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [94]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,5.020927e+07,...,1.921789e+10,8.202881e+10,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,247.134
1,2003-2,0.756708,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,5.018273e+07,...,1.921789e+10,8.222716e+10,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,305.708
2,2003-3,0.756879,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,5.016725e+07,...,1.921789e+10,8.234287e+10,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,284.168
3,2003-4,0.757050,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,5.016559e+07,...,1.921789e+10,8.235526e+10,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,286.594
4,2003-5,0.757222,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,5.011916e+07,...,1.921843e+10,8.245980e+10,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,559.950


In [95]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.968698,-1.575817,-2.202855,-0.752424,-1.806910,-1.971018,-3.111816,-2.777792,1.874594,-1.361502,...,1.155957,-1.063593,-0.822933,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.943192,-1.560482,-2.182122,-0.759292,-1.786501,-1.919572,-3.024730,-2.713279,1.869312,-1.359746,...,1.155957,-1.063593,-0.818174,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.917685,-1.548550,-2.161328,-0.766160,-1.766091,-1.868126,-2.937643,-2.648766,1.866230,-1.358722,...,1.155957,-1.063593,-0.815398,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.892178,-1.548848,-2.156390,-0.773028,-1.745682,-1.816680,-2.850557,-2.584252,1.865900,-1.358612,...,1.155957,-1.063593,-0.815101,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.866672,-1.550544,-2.136466,-0.779896,-1.725272,-1.765234,-2.763471,-2.519739,1.856658,-1.352530,...,1.151943,-1.063566,-0.812592,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241,-0.624727,1.575950,...,3.115817,1.690001,0.667867,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.064113,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578,-0.548742,1.764167,...,3.295619,1.717926,0.640959,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.046949,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914,-0.439805,1.909130,...,3.486686,1.748758,0.615514,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.029785,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250,-0.327203,2.040785,...,3.692016,1.780444,0.593546,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [96]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      305.708
1      284.168
2      286.594
3      312.455
4      299.651
        ...   
235    612.428
236    579.831
237    559.950
238    559.950
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [97]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
60,-0.617077,-0.757795,-0.238292,-1.028493,-0.822444,0.927636,0.275859,-0.396951,-0.253335,0.485766,...,-0.660781,-0.907428,-1.023454,-0.537241,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813
61,-0.598696,-0.727075,-0.209865,-1.025680,-0.813752,0.973169,0.285191,-0.394134,-0.296485,0.526179,...,-0.660781,-0.899853,-1.024085,-0.479316,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566
62,-0.580315,-0.690455,-0.153086,-1.022868,-0.805061,1.018702,0.294523,-0.391316,-0.338941,0.566133,...,-0.660781,-0.892039,-1.024307,-0.466312,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327
63,-0.561934,-0.651003,-0.114447,-1.020055,-0.796370,1.064234,0.303855,-0.388499,-0.380655,0.605591,...,-0.660781,-0.883981,-1.024104,-0.438778,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189
64,-0.543553,-0.607147,-0.062262,-1.017242,-0.787679,1.109767,0.313188,-0.385682,-0.421583,0.644515,...,-0.660781,-0.875677,-1.023461,-0.457839,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.334554,0.717612,-0.030629,1.239700,1.136971,-0.986304,0.021589,0.677638,-0.685481,-1.113174,...,-0.660781,1.272681,1.521503,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.337445,0.715300,-0.017187,1.195852,1.142243,-0.994112,0.041435,0.688796,-0.757697,-1.166440,...,-0.660781,1.288866,1.512161,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.340335,0.716462,-0.003286,1.152004,1.147515,-1.001919,0.061280,0.699954,-0.844729,-1.204659,...,-0.660781,1.304998,1.500026,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.343226,0.715461,0.000561,1.108156,1.152788,-1.009727,0.081125,0.711112,-0.914820,-1.250199,...,-0.660781,1.332157,1.486995,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [98]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60     402.782000
61     429.874000
62     484.463000
63     498.487000
64     506.647000
          ...    
163    411.310457
164    406.131343
165    400.952229
166    395.773114
167    390.594000
Name: Minas Gerais - Consumo de Cimento (t), Length: 108, dtype: float64

In [99]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
168,1.349007,0.715008,0.019780,1.020461,1.163332,-1.025342,0.120816,0.733427,-1.028195,-1.328800,...,-0.660781,1.364625,1.460212,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117
169,1.343125,0.715570,0.029139,1.001834,1.162557,-1.011264,0.132877,0.734965,-1.102985,-1.327468,...,-0.660781,1.371257,1.445627,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274
170,1.337243,0.716709,0.052997,0.983208,1.161781,-0.997185,0.144939,0.736502,-1.168851,-1.298530,...,-0.660781,1.375328,1.430008,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341
171,1.331361,0.718059,0.071070,0.964582,1.161006,-0.983107,0.157000,0.738040,-1.188592,-1.273097,...,-0.660781,1.381257,1.413730,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301
172,1.325480,0.719639,0.140932,0.945955,1.160230,-0.969029,0.169061,0.739577,-1.180922,-1.191250,...,-0.660781,1.364882,1.396980,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284
173,1.319598,0.721466,0.186483,0.927329,1.159455,-0.954950,0.181122,0.741115,-1.124581,-1.154162,...,-0.582593,1.369812,1.379116,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348
174,1.313716,0.723499,0.258333,0.908702,1.158679,-0.940872,0.193184,0.742652,-1.064934,-1.137769,...,-0.474908,1.376543,1.360141,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659
175,1.307834,0.725580,0.325035,0.890076,1.157904,-0.926794,0.205245,0.744190,-1.111858,-1.110088,...,-0.364364,1.384496,1.341361,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184
176,1.301952,0.727713,0.348449,0.871450,1.157128,-0.912715,0.217306,0.745727,-1.162547,-1.073320,...,-0.250935,1.391358,1.323058,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173
177,1.296071,0.729905,0.378109,0.852823,1.156353,-0.898637,0.229368,0.747265,-1.165964,-1.015391,...,-0.134596,1.399740,1.303196,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760


In [100]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    338.767
169    419.962
170    387.800
171    391.122
172    409.739
173    413.460
174    497.019
175    476.478
176    421.395
177    413.641
178    338.033
179    431.028
180    305.105
181    402.406
182    419.419
183    336.054
184    568.395
185    541.488
186    537.851
187    526.265
188    559.293
189    466.646
190    416.049
191    501.489
192    460.699
193    445.583
194    509.271
195    532.602
196    500.905
197    588.495
198    589.349
199    543.982
200    614.074
201    557.419
202    412.582
203    468.600
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [170]:
def validation_splitter(df, div_factor, add_factor):
    split_factor = len(df)//div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop_index.append(pos + add_factor)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop_index), df.iloc[positions_to_drop]

In [174]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 10, 60)
    target,target_val = validation_splitter(train_target, 10, 60)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [175]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [176]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2977877863, 3840246137, 687326389, 712764832, 4086670509, 1568353385, 907897041, 1828385246, 2871866516, 344300677, 4074682721, 1279155853, 2938768673, 3209441437, 2429227628, 1274879841, 4011410666, 3017892372, 3021276758, 3276064302, 3302184702, 1638179541, 4052134272, 3807402168, 1123523001]


Step: 0 ___________________________________________
val_loss: 687.4735107421875
winner_seed: 2977877863


Step: 1 ___________________________________________
val_loss: 696.9569091796875


Step: 2 ___________________________________________
val_loss: 708.2952880859375


Step: 3 ___________________________________________
val_loss: 673.3743286132812
winner_seed: 712764832


Step: 4 ___________________________________________
val_loss: 462.462646484375
winner_seed: 4086670509


Step: 5 ___________________________________________
val_loss: 533.6087646484375


Step: 6 ___________________________________________
val_loss: 285.2168884277344
winner_seed: 907897041


Step: 7 __________________________

In [177]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 0s 32ms/step - loss: 242983.4688 - val_loss: 192462.8750
Epoch 2/10000
4/4 [==============================] - 0s 5ms/step - loss: 307724.2188 - val_loss: 22393.4102
Epoch 3/10000
4/4 [==============================] - 0s 6ms/step - loss: 118236.3984 - val_loss: 243530.7969
Epoch 4/10000
4/4 [==============================] - 0s 5ms/step - loss: 107843.5078 - val_loss: 25270.6445
Epoch 5/10000
4/4 [==============================] - 0s 5ms/step - loss: 128884.8281 - val_loss: 142060.8438
Epoch 6/10000
4/4 [==============================] - 0s 6ms/step - loss: 138630.0781 - val_loss: 175183.4062
Epoch 7/10000
4/4 [==============================] - 0s 5ms/step - loss: 197401.8438 - val_loss: 638030.2500
Epoch 8/10000
4/4 [==============================] - 0s 6ms/step - loss: 191769.6094 - val_loss: 399632.4688
Epoch 9/10000
4/4 [==============================] - 0s 6ms/step - loss: 151600.3906 - val_loss: 115440.7891
Epoch 10/10000
4/4 [

4/4 [==============================] - 0s 6ms/step - loss: 8090.5503 - val_loss: 7428.6377
Epoch 78/10000
4/4 [==============================] - 0s 5ms/step - loss: 13855.4678 - val_loss: 14227.7080
Epoch 79/10000
4/4 [==============================] - 0s 6ms/step - loss: 11147.9473 - val_loss: 3946.6851
Epoch 80/10000
4/4 [==============================] - 0s 5ms/step - loss: 8427.3203 - val_loss: 2836.1079
Epoch 81/10000
4/4 [==============================] - 0s 5ms/step - loss: 9542.5254 - val_loss: 2456.0835
Epoch 82/10000
4/4 [==============================] - 0s 6ms/step - loss: 8224.0254 - val_loss: 8437.2939
Epoch 83/10000
4/4 [==============================] - 0s 6ms/step - loss: 6433.4043 - val_loss: 970.7780
Epoch 84/10000
4/4 [==============================] - 0s 6ms/step - loss: 5047.7476 - val_loss: 1615.1737
Epoch 85/10000
4/4 [==============================] - 0s 5ms/step - loss: 5994.3833 - val_loss: 2464.1899
Epoch 86/10000
4/4 [==============================] - 0s 5m

4/4 [==============================] - 0s 6ms/step - loss: 6181.4526 - val_loss: 5021.8564
Epoch 154/10000
4/4 [==============================] - 0s 6ms/step - loss: 5534.2354 - val_loss: 3427.0833
Epoch 155/10000
4/4 [==============================] - 0s 6ms/step - loss: 5563.8960 - val_loss: 3268.3528
Epoch 156/10000
4/4 [==============================] - 0s 5ms/step - loss: 4456.2002 - val_loss: 3112.5632
Epoch 157/10000
4/4 [==============================] - 0s 5ms/step - loss: 5992.1284 - val_loss: 2860.6433
Epoch 158/10000
4/4 [==============================] - 0s 6ms/step - loss: 4901.1133 - val_loss: 4195.6982
Epoch 159/10000
4/4 [==============================] - 0s 5ms/step - loss: 5163.6084 - val_loss: 18749.6055
Epoch 160/10000
4/4 [==============================] - 0s 6ms/step - loss: 10484.8721 - val_loss: 3705.5273
Epoch 161/10000
4/4 [==============================] - 0s 6ms/step - loss: 5230.0444 - val_loss: 3595.5903
Epoch 162/10000
4/4 [==============================

4/4 [==============================] - 0s 5ms/step - loss: 4000.1084 - val_loss: 9841.9873
Epoch 230/10000
4/4 [==============================] - 0s 5ms/step - loss: 6123.2290 - val_loss: 1782.6887
Epoch 231/10000
4/4 [==============================] - 0s 5ms/step - loss: 3626.5605 - val_loss: 1871.8893
Epoch 232/10000
4/4 [==============================] - 0s 5ms/step - loss: 2983.9353 - val_loss: 3405.2407
Epoch 233/10000
4/4 [==============================] - 0s 5ms/step - loss: 4232.1274 - val_loss: 2654.9385
Epoch 234/10000
4/4 [==============================] - 0s 5ms/step - loss: 3173.2373 - val_loss: 1731.9606
Epoch 235/10000
4/4 [==============================] - 0s 5ms/step - loss: 3134.3301 - val_loss: 5254.5503
Epoch 236/10000
4/4 [==============================] - 0s 5ms/step - loss: 3467.2471 - val_loss: 1694.9553
Epoch 237/10000
4/4 [==============================] - 0s 5ms/step - loss: 2933.0886 - val_loss: 2719.8511
Epoch 238/10000
4/4 [==============================] 

4/4 [==============================] - 0s 6ms/step - loss: 2725.2319 - val_loss: 3515.1687
Epoch 306/10000
4/4 [==============================] - 0s 6ms/step - loss: 3761.2598 - val_loss: 7548.5693
Epoch 307/10000
4/4 [==============================] - 0s 5ms/step - loss: 4775.9014 - val_loss: 2232.6868
Epoch 308/10000
4/4 [==============================] - 0s 6ms/step - loss: 2334.3582 - val_loss: 2660.0317
Epoch 309/10000
4/4 [==============================] - 0s 6ms/step - loss: 2467.1252 - val_loss: 2414.1653
Epoch 310/10000
4/4 [==============================] - 0s 5ms/step - loss: 3551.6294 - val_loss: 3412.2070
Epoch 311/10000
4/4 [==============================] - 0s 5ms/step - loss: 3580.3499 - val_loss: 12662.5137
Epoch 312/10000
4/4 [==============================] - 0s 5ms/step - loss: 5784.1143 - val_loss: 2604.0742
Epoch 313/10000
4/4 [==============================] - 0s 6ms/step - loss: 2812.9490 - val_loss: 2461.6804
Epoch 314/10000
4/4 [==============================]

...

4/4 [==============================] - 0s 5ms/step - loss: 5212.7876 - val_loss: 2748.0332
Epoch 1220/10000
4/4 [==============================] - 0s 6ms/step - loss: 3620.3955 - val_loss: 5332.5732
Epoch 1221/10000
4/4 [==============================] - 0s 6ms/step - loss: 3556.2993 - val_loss: 2935.0503
Epoch 1222/10000
4/4 [==============================] - 0s 5ms/step - loss: 3634.5095 - val_loss: 2930.4460
Epoch 1223/10000
4/4 [==============================] - 0s 5ms/step - loss: 3678.9773 - val_loss: 4212.0024
Epoch 1224/10000
4/4 [==============================] - 0s 5ms/step - loss: 3587.1936 - val_loss: 3018.6094
Epoch 1225/10000
4/4 [==============================] - 0s 5ms/step - loss: 3663.6616 - val_loss: 3241.9297
Epoch 1226/10000
4/4 [==============================] - 0s 6ms/step - loss: 3368.5574 - val_loss: 2931.4309
Epoch 1227/10000
4/4 [==============================] - 0s 6ms/step - loss: 3537.8979 - val_loss: 4256.4014
Epoch 1228/10000
4/4 [=======================

Epoch 1295/10000
4/4 [==============================] - 0s 5ms/step - loss: 3796.8020 - val_loss: 3874.2319
Epoch 1296/10000
4/4 [==============================] - 0s 5ms/step - loss: 3527.7305 - val_loss: 3854.5735
Epoch 1297/10000
4/4 [==============================] - 0s 6ms/step - loss: 4154.0615 - val_loss: 2946.8491
Epoch 1298/10000
4/4 [==============================] - 0s 5ms/step - loss: 3812.3865 - val_loss: 2182.7231
Epoch 1299/10000
4/4 [==============================] - 0s 5ms/step - loss: 3664.3330 - val_loss: 2077.0803
Epoch 1300/10000
4/4 [==============================] - 0s 5ms/step - loss: 3361.2793 - val_loss: 1639.2764
Epoch 1301/10000
4/4 [==============================] - 0s 5ms/step - loss: 3170.3145 - val_loss: 4735.1616
Epoch 1302/10000
4/4 [==============================] - 0s 5ms/step - loss: 6113.7559 - val_loss: 16445.0898
Epoch 1303/10000
4/4 [==============================] - 0s 5ms/step - loss: 6548.0376 - val_loss: 7301.3657
Epoch 1304/10000
4/4 [=====

Epoch 1371/10000
4/4 [==============================] - 0s 6ms/step - loss: 2559.9229 - val_loss: 3618.1001
Epoch 1372/10000
4/4 [==============================] - 0s 5ms/step - loss: 2881.6592 - val_loss: 4472.4082
Epoch 1373/10000
4/4 [==============================] - 0s 5ms/step - loss: 2781.2991 - val_loss: 12917.8809
Epoch 1374/10000
4/4 [==============================] - 0s 5ms/step - loss: 3903.3145 - val_loss: 5314.1680
Epoch 1375/10000
4/4 [==============================] - 0s 5ms/step - loss: 3163.9856 - val_loss: 3455.5981
Epoch 1376/10000
4/4 [==============================] - 0s 5ms/step - loss: 3600.6875 - val_loss: 40460.8672
Epoch 1377/10000
4/4 [==============================] - 0s 5ms/step - loss: 13337.0117 - val_loss: 3371.8730
Epoch 1378/10000
4/4 [==============================] - 0s 5ms/step - loss: 3984.0085 - val_loss: 4000.4929
Epoch 1379/10000
4/4 [==============================] - 0s 5ms/step - loss: 3508.4438 - val_loss: 3865.0171
Epoch 1380/10000
4/4 [===

Epoch 1447/10000
4/4 [==============================] - 0s 5ms/step - loss: 3373.1426 - val_loss: 3267.9097
Epoch 1448/10000
4/4 [==============================] - 0s 5ms/step - loss: 3554.0342 - val_loss: 2880.3394
Epoch 1449/10000
4/4 [==============================] - 0s 5ms/step - loss: 3548.3425 - val_loss: 3102.3184
Epoch 1450/10000
4/4 [==============================] - 0s 6ms/step - loss: 3544.3389 - val_loss: 2981.0300
Epoch 1451/10000
4/4 [==============================] - 0s 5ms/step - loss: 3775.2810 - val_loss: 3343.3762
Epoch 1452/10000
4/4 [==============================] - 0s 5ms/step - loss: 3459.1987 - val_loss: 5189.1221
Epoch 1453/10000
4/4 [==============================] - 0s 6ms/step - loss: 3733.5198 - val_loss: 4041.9980
Epoch 1454/10000
4/4 [==============================] - 0s 6ms/step - loss: 3635.7085 - val_loss: 3134.4937
Epoch 1455/10000
4/4 [==============================] - 0s 5ms/step - loss: 4057.5364 - val_loss: 1994.8356
Epoch 1456/10000
4/4 [======

Epoch 1523/10000
4/4 [==============================] - 0s 5ms/step - loss: 3150.9866 - val_loss: 3429.6257
Epoch 1524/10000
4/4 [==============================] - 0s 5ms/step - loss: 2878.4695 - val_loss: 1901.9098
Epoch 1525/10000
4/4 [==============================] - 0s 5ms/step - loss: 3082.4832 - val_loss: 1796.9519
Epoch 1526/10000
4/4 [==============================] - 0s 5ms/step - loss: 2519.9419 - val_loss: 2671.4155
Epoch 1527/10000
4/4 [==============================] - 0s 6ms/step - loss: 3096.6694 - val_loss: 1822.9626
Epoch 1528/10000
4/4 [==============================] - 0s 6ms/step - loss: 3540.4082 - val_loss: 7389.3877
Epoch 1529/10000
4/4 [==============================] - 0s 5ms/step - loss: 4916.4795 - val_loss: 1732.6218
Epoch 1530/10000
4/4 [==============================] - 0s 5ms/step - loss: 2859.7759 - val_loss: 1735.3779
Epoch 1531/10000
4/4 [==============================] - 0s 6ms/step - loss: 2959.7310 - val_loss: 2155.8779
Epoch 1532/10000
4/4 [======

In [178]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [179]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 34ms/step
Month-1 - Error: [[77.07605]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[4.1189575]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[28.04306]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[24.721039]]
1/1 [==============================] - 0s 14ms/step
Month-5 - Error: [[6.1040344]]
1/1 [==============================] - 0s 13ms/step
Month-6 - Error: [[2.3830566]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[81.175964]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[60.63495]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[5.551941]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[2.202057]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[77.81006]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[15.184967]]
1/1 [==

In [180]:
display(mae)
display(mape)

72.86789

0.14836809

In [181]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [182]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[4990.117]] - Target[4938.444]| =  Error: [[51.67334]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[4990.117]] - Target[5580.459999999999]| =  Error: [[590.3428]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[4990.117]] - Target[6223.561000000001]| =  Error: [[1233.4438]]


[array([[51.67334]], dtype=float32),
 array([[590.3428]], dtype=float32),
 array([[1233.4438]], dtype=float32)]

625.1533

0.10481346